In [ ]:
import json
import multiprocessing
import pandas as pd
import pickle
import spotipy
import time

from spotipy.oauth2 import SpotifyClientCredentials

In [ ]:
artists_names = list(pd.read_csv('artists_names.csv')['artist'])
print(artists_names[:20])

In [ ]:
# Utilizando o Spotipy

CLIENT_ID = 'facc2f06aabf418b8b2503349baec6ca'
CLIENT_SECRET = 'e11a217cf7f54674a625122ba3e5c17e'

client_credentials = SpotifyClientCredentials(client_id=CLIENT_ID, client_secret=CLIENT_SECRET)
spotify = spotipy.Spotify(auth_manager=client_credentials)

In [ ]:
# Método para obter todos os tipos de álbuns de um artista dado o ID do artista
def get_all_albums_of_artist(artist_id):
    artist = spotify.artist(artist_id)    
    all_albums = []
    album_types = ['album', 'single']

    for album_type in album_types:
        offset = 0

        while True:
            albums = spotify.artist_albums(artist_id, album_type=album_type, limit=50, offset=offset)

            if not albums['items']:
                break

            all_albums.extend(albums['items'])
            offset += 50

    return all_albums

def get_all_album_tracks(album_id):
    album = spotify.album_tracks(album_id)

    return album['items']

# Get the artist
def get_first_artist(artist_name, json_response):
    artists = json_response['artists']['items']
    
    if not artists:
        raise Exception('Artista não encontrado')
    
    for artist in artists:
        if artist['name'].strip().lower() == artist_name.lower():
            return artist
    
    first_artist = artists[0]
    return first_artist

In [ ]:
musics = carregar_dicionario('musics.pkl')
start_index = carregar_numero_inteiro('musics_start_index.txt')

for i in range(start_index, len(artists_names)):
    artist_name = artists_names[i]

    results = spotify.search(q = artist_name, type = "artist", limit=50)
    first_artist = get_first_artist(artist_name, results)
    artist_id = first_artist['id']

    artist_response = spotify.artist(artist_id)
    artist_original_name = artist_name
    artist_spotify_name = artist_response['name']
    artist_genres = artist_response['genres']

    albums = get_all_albums_of_artist(artist_id)
    for album in albums:
        album_id = album['id']
        album_name = album['name']
        album_release_date = album['release_date']

        album_tracks = spotify.album_tracks(album_id)['items']
        for track in album_tracks:
            track_id = track['id']
            track_name = track['name']
            track_duration = track['duration_ms']

            musics['artist_original_name'].append(artist_original_name)
            musics['artist_spotify_name'].append(artist_spotify_name)
            musics['artist_id'].append(artist_id)
            musics['artist_genres'].append(artist_genres)
            musics['track_name'].append(track_name)
            musics['track_id'].append(track_id)
            musics['track_duration'].append(track_duration)
            musics['album_name'].append(album_name)
            musics['album_id'].append(album_id)
            musics['album_release_date'].append(album_release_date)

    salvar_dicionario(musics, 'musics.pkl')
    salvar_numero_inteiro(i + 1, 'musics_start_index.txt')
    print(i)'''

In [ ]:
musics = carregar_dicionario('musics.pkl')
musics_df = pd.DataFrame.from_dict(musics)
musics_df.to_csv('musics.csv', index = False)

musics_df